In [ ]:
#default_exp inference.text

# text.inference

> Provides inference scripts specifically for text modules

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai2.text.all import *
from fastinference.inference.inference import _fully_decode, _decode_loss
import matplotlib.cm as cm
import html
from IPython.display import display, HTML

In [ ]:
#export
@patch
def get_preds(x:LMLearner, ds_idx=1, dl=None, raw_outs=False, decoded_loss=True, fully_decoded=False, concat_dim=0,
             **kwargs):
    "Get predictions with possible decoding"
    inps, outs, dec_out, raw = [], [], [], []
    if dl is None: dl = x.dls[ds_idx].new(shuffle=False, drop_last=False)
    x.model.eval()
    for batch in dl:
        with torch.no_grad():
            inps.append(batch[:x.dls.n_inp])
            if decoded_loss or fully_decoded:
                out = x.model(*batch[:x.dls.n_inp])[0]
                raw.append(out)
                dec_out.append(x.loss_func.decodes(out))
            else:
                raw.append(x.model(*batch[:x.dls.n_inp])[0])
    raw = torch.cat(raw, dim=concat_dim).cpu().numpy()
    if decoded_loss or fully_decoded:
        dec_out = torch.cat(dec_out, dim=0)
    if not raw_outs:
        try: outs.insert(0, x.loss_func.activation(tensor(raw)).numpy())
        except: outs.insert(0, dec_out)
    else:
        outs.insert(0, raw)
    if fully_decoded: outs = _fully_decode(x.dls, inps, outs, dec_out, False)
    if decoded_loss: outs = _decode_loss(x.dls.categorize.vocab, dec_out, outs)
    return outs

In [ ]:
show_doc(LMLearner.get_preds)

<h4 id="LMLearner.get_preds" class="doc_header"><code>LMLearner.get_preds</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>LMLearner.get_preds</code>(**`x`**:`LMLearner`, **`ds_idx`**=*`1`*, **`dl`**=*`None`*, **`raw_outs`**=*`False`*, **`decoded_loss`**=*`True`*, **`fully_decoded`**=*`False`*, **`concat_dim`**=*`0`*, **\*\*`kwargs`**)

Get predictions with possible decoding

In [ ]:
#export
@patch
def get_preds(x:TextLearner, ds_idx=1, dl=None, raw_outs=False, decoded_loss=True, fully_decoded=False,
             **kwargs):
    "Get predictions with possible decoding"
    inps, outs, dec_out, raw = [], [], [], []
    if dl is None: dl = x.dls[ds_idx].new(shuffle=False, drop_last=False)
    x.model.eval()
    for batch in dl:
        with torch.no_grad():
            inps.append(batch[:x.dls.n_inp])
            if decoded_loss or fully_decoded:
                out = x.model(*batch[:x.dls.n_inp])[0]
                raw.append(out)
                dec_out.append(x.loss_func.decodes(out))
            else:
                raw.append(x.model(*batch[:x.dls.n_inp])[0])
    raw = torch.cat(raw, dim=0).cpu().numpy()
    if decoded_loss or fully_decoded:
        dec_out = torch.cat(dec_out, dim=0)
    if not raw_outs:
        try: outs.insert(0, x.loss_func.activation(tensor(raw)).numpy())
        except: outs.insert(0, dec_out)
    else:
        outs.insert(0, raw)
    if fully_decoded: outs = _fully_decode(x.dls, inps, outs, dec_out, False)
    if decoded_loss: outs = _decode_loss(x.dls.categorize.vocab, dec_out, outs)
    return outs

In [ ]:
show_doc(TextLearner.get_preds)

<h4 id="Learner.get_preds" class="doc_header"><code>Learner.get_preds</code><a href="https://github.com/muellerzr/fastinference/tree/master/fastinference/inference/inference.py#L39" class="source_link" style="float:right">[source]</a></h4>

> <code>Learner.get_preds</code>(**`x`**:`Learner`, **`ds_idx`**=*`1`*, **`dl`**=*`None`*, **`raw_outs`**=*`False`*, **`decoded_loss`**=*`True`*, **`fully_decoded`**=*`False`*, **\*\*`kwargs`**)

Get predictions with possible decoding

In [ ]:
#export
@patch
def predict(x:LMLearner, text, n_words=1, no_unk=True, temperature=1., min_p=None, 
                decoder=decode_spec_tokens, only_last_word=False):
        "Predict `n_words` from `text`"
        x.model.reset()
        idxs = idxs_all = x.dls.test_dl([text]).items[0].to(x.dls.device)
        unk_idx = x.dls.vocab.index(UNK)
        for _ in (range(n_words)):
            preds = x.get_preds(dl=[(idxs[None],)], decoded_loss=False)
            res = preds[0][0][-1]
            if no_unk: res[unk_idx] = 0.
            if min_p is not None:
                if (res >= min_p).float().sum() == 0:
                    warn(f"There is no item with probability >= {min_p}, try a lower value.")
                else: res[res < min_p] = 0.
            if temperature != 1.: res.pow_(1 / temperature)
            res = tensor(res)
            idx = torch.multinomial(res, 1).item()
            idxs = idxs_all = torch.cat([idxs_all, idxs.new([idx])])
            if only_last_word: idxs = idxs[-1][None]
        decoder=decode_spec_tokens
        num = x.dls.train_ds.numericalize
        tokens = [num.vocab[i] for i in idxs_all if num.vocab[i] not in [BOS, PAD]]
        sep = x.dls.train_ds.tokenizer[-1].sep
        return sep.join(decoder(tokens))

In [ ]:
show_doc(LMLearner.predict)

<h4 id="LMLearner.predict" class="doc_header"><code>LMLearner.predict</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>LMLearner.predict</code>(**`x`**:`LMLearner`, **`text`**, **`n_words`**=*`1`*, **`no_unk`**=*`True`*, **`temperature`**=*`1.0`*, **`min_p`**=*`None`*, **`decoder`**=*`'decode_spec_tokens'`*, **`only_last_word`**=*`False`*)

Predict `n_words` from `text`

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')
data_lm = TextDataLoaders.from_csv(path, 'texts.csv', text_col='text', is_lm=True)
lm_learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
lm_learn.save_encoder('fine_tuned')
blocks = (TextBlock.from_df('text', seq_len=data_lm.seq_len, vocab=data_lm.vocab), CategoryBlock())
imdb_clas = DataBlock(blocks=blocks,
                      get_x=ColReader('text'),
                      get_y=ColReader('label'),
                      splitter=ColSplitter())
dls = imdb_clas.dataloaders(df, bs=64)


In [ ]:
lm_learn.predict('my name is', n_words=2)

'my name is not always'

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.path = path
learn.load_encoder('fine_tuned');

In [ ]:
dl = learn.dls.test_dl(df.iloc[:1])

In [ ]:
name, probs, full_dec = learn.get_preds(dl=dl, fully_decoded=True)

In [ ]:
name, probs

(['negative'], array([[0.5106333 , 0.48936665]], dtype=float32))

In [ ]:
full_dec

(#1) [("xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk … xxmaj xxunk xxrep 3 ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . w xxrep 3 o xxrep 3 f !", 'negative')]

In [ ]:
#export
def _value2rgba(x, cmap=cm.RdYlGn, alpha_mult=1.0):
    "Convert a value `x` from 0 to 1 (inclusive) to an RGBA tuple according to `cmap` times transparency `alpha_mult`."
    c = cmap(x)
    rgb = (np.array(c[:-1]) * 255).astype(int)
    a = c[-1] * alpha_mult
    return tuple(rgb.tolist() + [a])

In [ ]:
#export
def _eval_dropouts(mod):
        module_name =  mod.__class__.__name__
        if 'Dropout' in module_name or 'BatchNorm' in module_name: mod.training = False
        for module in mod.children(): _eval_dropouts(module)

In [ ]:
#export
def _piece_attn_html(pieces, attns, sep=' ', **kwargs):
    html_code,spans = ['<span style="font-family: monospace;">'], []
    for p, a in zip(pieces, attns):
        p = html.escape(p)
        c = str(_value2rgba(a, alpha_mult=0.5, **kwargs))
        spans.append(f'<span title="{a:.3f}" style="background-color: rgba{c};">{p}</span>')
    html_code.append(sep.join(spans))
    html_code.append('</span>')
    return ''.join(html_code)

def _show_piece_attn(*args, **kwargs):
    from IPython.display import display, HTML
    display(HTML(_piece_attn_html(*args, **kwargs)))

In [ ]:
#export
def _intrinsic_attention(learn, text, class_id=None):
    "Calculate the intrinsic attention of the input w.r.t to an output `class_id`, or the classification given by the model if `None`."
    learn.model.train()
    _eval_dropouts(learn.model)
    learn.model.zero_grad()
    learn.model.reset()
    dl = learn.dls.test_dl([text])
    batch = next(iter(dl))[0]
    emb = learn.model[0].module.encoder(batch).detach().requires_grad_(True)
    lstm = learn.model[0].module(emb, True)
    learn.model.eval()
    cl = learn.model[1]((lstm, torch.zeros_like(batch).bool(),))[0].softmax(dim=-1)
    if class_id is None: class_id = cl.argmax()
    cl[0][class_id].backward()
    attn = emb.squeeze().abs().sum(dim=-1)
    attn /= attn.max()
    tok, _ = learn.dls.decode_batch((*tuplify(batch), *tuplify(cl)))[0]
    return tok, attn

In [ ]:
#export
@patch
def intrinsic_attention(x:TextLearner, text:str, class_id:int=None, **kwargs):
    "Shows the `intrinsic attention for `text`, optional `class_id`"
    if isinstance(x, LMLearner): raise Exception("Language models are not supported")
    text, attn = _intrinsic_attention(x, text, class_id)
    return _show_piece_attn(text.split(), to_np(attn), **kwargs)

In [ ]:
show_doc(TextLearner.intrinsic_attention)

<h4 id="TextLearner.intrinsic_attention" class="doc_header"><code>TextLearner.intrinsic_attention</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>TextLearner.intrinsic_attention</code>(**`x`**:`TextLearner`, **`text`**:`str`, **`class_id`**:`int`=*`None`*, **\*\*`kwargs`**)

Shows the `intrinsic attention for `text`, optional `class_id`

In [ ]:
learn.intrinsic_attention('Batman is rich')